# check torch

In [98]:
import torch

print("CUDA available?", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


CUDA available? True
Device count: 1
Device name: NVIDIA GeForce RTX 4070 SUPER


In [99]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


# Prepare Dataset

In [100]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import pandas as pd
import os
df = pd.read_pickle('data/trainingData.pkl')
miete_col = 'wohn_leer'

df = df.drop(columns=[miete_col])
X = df.drop(columns=['preis_miet_best'], axis=1).values
y = df['preis_miet_best'].values


In [115]:

# Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)  # use float32 for regression targets
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)


#  Create Dataset and DataLoader

In [116]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)


# Define the Model Class

In [117]:
import torch.nn as nn


class TabularNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout_rate, activation_fn):
        super(TabularNN, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            if activation_fn == 'relu':
                layers.append(nn.ReLU())
            elif activation_fn == 'tanh':
                layers.append(nn.Tanh())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 1))  # output 1 value for regression
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).squeeze(1)  # output shape: (batch_size,)



#  Define Training and Evaluation Functions

In [120]:
def train_one_epoch(model, optimizer, criterion, dataloader, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.float().to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    return total_loss / len(dataloader.dataset)

def evaluate(model, dataloader, device):
    model.eval()
    criterion = torch.nn.MSELoss(reduction="sum")  # Sum so we can compute RMSE ourselves
    total_squared_error = 0.0
    total_samples = 0

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.float().to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_squared_error += loss.item()
            total_samples += y_batch.size(0)

    mse = total_squared_error / total_samples
    rmse = mse ** 0.5
    return rmse



# Define the Optuna Objective Function

In [121]:
import optuna

def objective(trial):
    try:
        # sample hyperparams
        hidden_dim1 = trial.suggest_int("hidden_dim1", 32, 256)
        hidden_dim2 = trial.suggest_int("hidden_dim2", 32, 256)
        dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
        lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
        batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
        activation_fn = trial.suggest_categorical("activation_fn", ["relu", "tanh"])

        model = TabularNN(
            input_dim=100,
            hidden_dims=[hidden_dim1, hidden_dim2],
            dropout_rate=dropout_rate,
            activation_fn=activation_fn,
        ).to(device)

        # DataLoader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = torch.nn.MSELoss()

        for epoch in range(10):
            train_one_epoch(model, optimizer, criterion, train_loader, device)
            val_rmse = evaluate(model, val_loader, device)
            trial.report(val_rmse, epoch)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return val_rmse

    except Exception as e:
        print(f"Trial failed with exception: {e}")
        raise  # Reraise so Optuna logs the failure



# finer 

In [95]:
def objective(trial):
    try:
        hidden_dim1 = trial.suggest_int("hidden_dim1", 100, 130)
        hidden_dim2 = trial.suggest_int("hidden_dim2", 98, 128)
        dropout_rate = trial.suggest_float("dropout_rate", 0.01, 0.05)
        lr = trial.suggest_float("lr", 1e-3, 3e-3, log=True)
        batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
        activation_fn = trial.suggest_categorical("activation_fn", ["relu", "tanh"])

        model = TabularNN(
            input_dim=100,
            hidden_dims=[hidden_dim1, hidden_dim2],
            dropout_rate=dropout_rate,
            activation_fn=activation_fn,
        ).to(device)

        # DataLoader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = torch.nn.MSELoss()

        for epoch in range(10):
            train_one_epoch(model, optimizer, criterion, train_loader, device)
            val_rmse = evaluate(model, val_loader, device)
            trial.report(val_rmse, epoch)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return val_rmse

    except Exception as e:
        print(f"Trial failed with exception: {e}")
        raise  # Reraise so Optuna logs the failure



# Launch the Optimization

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)


[I 2025-07-08 21:05:34,949] A new study created in memory with name: no-name-d7b86a7e-8de3-4a9f-829d-5a00717bd6fb
[I 2025-07-08 21:05:38,269] Trial 0 finished with value: 0.5974501145356033 and parameters: {'hidden_dim1': 137, 'hidden_dim2': 56, 'dropout_rate': 0.052712137844521556, 'lr': 0.00804896618267645, 'batch_size': 32, 'activation_fn': 'relu'}. Best is trial 0 with value: 0.5974501145356033.
[I 2025-07-08 21:05:39,358] Trial 1 finished with value: 0.8023553786641573 and parameters: {'hidden_dim1': 123, 'hidden_dim2': 240, 'dropout_rate': 0.2985756927078828, 'lr': 0.0008548820788328283, 'batch_size': 128, 'activation_fn': 'relu'}. Best is trial 0 with value: 0.5974501145356033.
[I 2025-07-08 21:05:41,140] Trial 2 finished with value: 0.4793029590149357 and parameters: {'hidden_dim1': 70, 'hidden_dim2': 233, 'dropout_rate': 0.04160877144799341, 'lr': 0.00227296157255192, 'batch_size': 64, 'activation_fn': 'tanh'}. Best is trial 2 with value: 0.4793029590149357.
[I 2025-07-08 21:0

Trial failed with exception: 


[I 2025-07-08 21:05:45,556] Trial 6 pruned. 


Trial failed with exception: 


[I 2025-07-08 21:05:49,052] Trial 7 finished with value: 0.5368477700756129 and parameters: {'hidden_dim1': 50, 'hidden_dim2': 138, 'dropout_rate': 0.08314526874901257, 'lr': 0.00981854412155033, 'batch_size': 32, 'activation_fn': 'relu'}. Best is trial 2 with value: 0.4793029590149357.
[I 2025-07-08 21:05:49,445] Trial 8 pruned. 


...

[I 2025-07-08 21:06:37,634] Trial 51 pruned. 


Trial failed with exception: 


[I 2025-07-08 21:06:37,964] Trial 52 pruned. 


Trial failed with exception: 


[I 2025-07-08 21:06:41,268] Trial 53 finished with value: 0.47458772048052317 and parameters: {'hidden_dim1': 89, 'hidden_dim2': 66, 'dropout_rate': 0.04323331497077106, 'lr': 0.0067325783199970665, 'batch_size': 32, 'activation_fn': 'tanh'}. Best is trial 53 with value: 0.47458772048052317.


In [114]:
print(X_train_tensor.device)

cpu


#  Print and Save Best Result

In [ ]:
print("Best hyperparameters:", study.best_params)
print(f"Best validation accuracy: {1.0 - study.best_value:.4f}")
best_params = study.best_params
best_model = TabularNN(
    input_dim=100,
    hidden_dims=[best_params["hidden_dim1"], best_params["hidden_dim2"]],
    dropout_rate=best_params["dropout_rate"],
    activation_fn=best_params["activation_fn"],
).to(device)

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params["batch_size"], shuffle=False)

optimizer = torch.optim.Adam(best_model.parameters(), lr=best_params["lr"])
criterion = nn.MSELoss()
# Retrain
for epoch in range(10):
    train_one_epoch(best_model, optimizer, criterion, train_loader, device)

#torch.save(best_model.state_dict(), "best_tabular_model.pt")


Best hyperparameters: {'hidden_dim1': 108, 'hidden_dim2': 110, 'dropout_rate': 0.013793323529972864, 'lr': 0.0029455288566947136, 'batch_size': 32, 'activation_fn': 'tanh'}
Best validation accuracy: 0.4746


In [97]:
print("torch.cuda.is_available():", torch.cuda.is_available())

# After creating the model
print(next(model.parameters()).device)

# In training loop, after moving to device
print(X_batch.device)
print(y_batch.device)


torch.cuda.is_available(): True


NameError: name 'model' is not defined